In [1]:
#Import of Python available libraries
import sculib

mpi = sculib.scu()
mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '10.96.254.85'

In [2]:
def mpi_t622(scu):

    #Tilt sensor Henk email 3) night time 
    print('Test 6.2.2')

    #Revisions:
    #0.1 initial write - could do with some kat sensor retrieval
    #0.2 rewrite for sculib

    #Common setup
    config_name = 'HN_TILT_TEST'

    
    loop1 = 10
    azimuths = [
        -200, -180, -170, -160, -150, -140, -130, -120, -110, -100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0,
        10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 200]
    az_start = -180
    az_speed = 3
    az_speed_tilt = 0.2

    #elevations = [85, 50, 15] lower limit not accessible
    #elevations = [85, 50, 20]
    elevations = [20, 45, 85] 
    el_speed  = 0.1
    
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #debug test only remove these
    '''
    loop1 = 4
    elevations = [20]

    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug = False
    '''

    #B. Tilt Sensor tests [316-000000-043]
    '''
    a. Tilt Sensor: Offset calibration & stability (6.2.2)
    1) Manually move the Dish to 85° Elevation, -180° Azimuth and de-activate all axes
    2) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)
    3) Activate the Azimuth and Elevation axis,
        ◦ wait for 15 seconds to complete the activation process
    4) Start the ACU data recording on the SCU using logging configuration TBD,
    5) Command Azimuth in 10° steps at 3°/sec (-180/-170…+170/+180), 
        ◦ wait 30 seconds between each step 
    6) Command Azimuth to 200° at 3deg/sec,
        ◦ wait for 30 sec
    7) Command Azimuth to -200° at 0.2°/sec,
        ◦  wait for 1030 sec
    8) Command Azimuth to 200° at 0.2°/sec,
        ◦ wait for 1030 sec
    9) Repeat steps 5) to 8), each cycle alternating the direction, for a total of 5 cycles
    10) Stop the ACU data recording on the SCU
    11) Deactivate the Azimuth and Elevation axis
    12) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)
    13) Extract the data on the SCU @ 5Hz sampling
    14) Rename the file to TBD (eg. 6p2p2_i.csv)
    15) Download the MeerKAT weather station data for the test period
    16) Repeat steps 1) to 15) for Elevation angle of 50°
    17) Repeat steps 1) to 15) for Elevation angle of 15°
    18) Repeat the test for 2 (TBC) additional nights (only/at least one Elevation angle)

    (Total duration ~4h30min per Elevation angle => 13h30min: start after 09h00 in evening through the night)
    '''
    #Step 1
    scu.activate_dmc()
    scu.wait_duration(wait10*3)

    #Move to starting az
    scu.abs_azimuth(az_start, az_speed)
    scu.wait_duration(wait100)

    for el_angle in elevations:
        #Move to starting el
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait100*5)

        #Step2
        print('Record pedestal wall temperature')

        #Step 3
        #Activate az el

        #Step 4

        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait10)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait10)

        i=0
        while i < loop1:
            i+=1
            print('Loop 1 - Elevation {:.0f} Step 5 to 9: {:.0f} of {:.0f} '.format(el_angle, i, loop1,))
            #Step 5 - Command Azimuth in 10° steps at 3°/sec (-180/-170…+170/+180),  #◦ wait 30 seconds 
            az_speed = 3
            for az_angle in azimuths[1:-1]:
                scu.abs_azimuth(az_angle, az_speed)
                scu.wait_duration(wait10*3)

            #Step 6
            #az_angle = 200
            az_speed = 3
            az_angle = azimuths[-1]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(wait10*3)


            #Step 7
            az_speed = az_speed_tilt
            #az_angle = -200
            az_angle = azimuths[0]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(wait100*10)
            scu.wait_duration(wait10*3)

            #Step 8
            az_speed = az_speed_tilt
            #az_angle = 200
            az_angle = azimuths[-1]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(wait100*10)
            scu.wait_duration(wait10*3)

            #Step9 repeat 5 to 8 x5 alternating directions
            azimuths = list(reversed(azimuths))

        #Step 10
        #stop recording
        stop_logger()
        scu.wait_duration(wait10)
        print(scu.logger_state())

        #Step 11


        #Step 12
        print('Record pedestal wall temperature')

        #Step 13
        print("Extract the data on the SCU @ 5Hz sampling")

        #Step 14
        print("Rename the file to TBD (eg.6p2p2_i.csv)")
        save_session('6p2p2_i_elev_' + str(el_angle), interval_ms = sampling)

        #Step 15
        print("Download the MeerKAT weather station data for the test period")
        #use cam sensors?

    #Step 16
    #16) Repeat steps 1) to 15) for Elevation angle of 50°
    #17) Repeat steps 1) to 15) for Elevation angle of 15°
    deactivate_dmc()
    scu.wait_duration(wait10*2)    


In [3]:
mpi_t622(mpi)
#mpi_t622(sim)


Test 6.2.2
wait for 3.0s done *
abs az: -180.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 5.0s done *
Record pedestal wall temperature
logger state 
WARNING, already recording - attempting to stop and start a fresh logger
wait for 1.0s done *
logger state 
Loop 1 - Elevation 20 Step 5 to 9: 1 of 4 
abs az: -180.000 vel: 3.000
wait for 3.0s done *
abs az: -170.000 vel: 3.000
wait for 3.0s done *
abs az: -160.000 vel: 3.000
wait for 3.0s done *
abs az: -150.000 vel: 3.000
wait for 3.0s done *
abs az: -140.000 vel: 3.000
wait for 3.0s done *
abs az: -130.000 vel: 3.000
wait for 3.0s done *
abs az: -120.000 vel: 3.000
wait for 3.0s done *
abs az: -110.000 vel: 3.000
wait for 3.0s done *
abs az: -100.000 vel: 3.000
wait for 3.0s done *
abs az: -90.000 vel: 3.000
wait for 3.0s done *
abs az: -80.000 vel: 3.000
wait for 3.0s done *
abs az: -70.000 vel: 3.000
wait for 3.0s done *
abs az: -60.000 vel: 3.000
wait for 3.0s done *
abs az: -50.000 vel: 3.000
wait for 3.0s 

wait for 10.0s done *
wait for 3.0s done *
abs az: -200.000 vel: 0.200
wait for 10.0s done *
wait for 3.0s done *
logger state 
RECORDING
Record pedestal wall temperature
Extract the data on the SCU @ 5Hz sampling
Rename the file to TBD (eg.6p2p2_i.csv)
Download the MeerKAT weather station data for the test period
